In [1]:
# scientific libraries
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
from skimage import io
from scipy.stats import circvar, circmean, circstd

# general libraries
import os  
import json
import time
import h5py as h5py
import copy
from functools import partial

# jupyter imports
import ipywidgets as widgets
from IPython.display import clear_output

# local libraries
from analysis import *
from general_utils import *
from hdf_utils import unpack_hdf, Workspace, pack_hdf
from scalebars import *
import deconv

import sac_mode_configs as configs
import experiments as exps
import analysis as ana
from interactive import MotionResponse

In [2]:
%matplotlib widget
plt.rcParams.update({"figure.max_open_warning": 0})
plt.rcParams['svg.fonttype'] = 'none'  # don't save font as paths (assumes font present on machine)

In [3]:
base_path = "/mnt/Data/NEURONoutput/sac_net/"

In [19]:
data_base_paths = {
    "vc": os.path.join(base_path, "vc_runs"),
    "spk": os.path.join(base_path, "model_runs"),
}
fig_path = os.path.join(base_path, "spk_vc_seed_sanity")
fig_subdir = None

fig_subdir, analysis_names = (None, {
    "vc": "poissarma_etau3_x0p45w_itau6_x0p8w_ar_0p9_-0p1_rho95flat_er1p2_ir0p6_2n_2tr_fixedrng_tree",
    "spk": "poissarma_etau3_x0p45w_itau6_x0p8w_ntau6_x0p3_ar0p9_-0p1_rho95_er1p2_ir0p6_nr1p2_2n_2tr_fixedrng_tree",
})

fig_path = fig_path if fig_subdir is None else os.path.join(fig_path, fig_subdir)
analysis_hdfs = {
    k: h5.File(os.path.join(data_base_paths[k], n + ".h5"), "r")
    for k, n in analysis_names.items()
}
d = {k: Workspace(hdf, read_only=True) for k, hdf in analysis_hdfs.items()}
# os.makedirs(fig_path, exist_ok=True)

rhos = list(d["vc"].keys())
idxs = list(d["vc"][rhos[0]].keys())
print("==keys==")
print("rhos:", rhos)
print("idxs:", idxs)
print("exp keys:", list(d["vc"][rhos[0]][idxs[0]].keys()))
print("n_syn:", len(d["spk"][rhos[0]][idxs[0]]["sac_net"]["wiring"]["gaba_here"][:]))

==keys==
rhos: [0.0, 1.0]
idxs: [0, 1]
exp keys: ['dendrites', 'params', 'sac_net', 'soma', 'syn_locs']
n_syn: 341


In [5]:
def close_button():
    button = widgets.Button(description="Close Workspace")
    out = widgets.Output()

    def on_clicked(_):
        with out:
            clear_output()
            for wrk in d.values():
                wrk.close()
            print("File closed!")
        
    button.on_click(on_clicked)
    return widgets.VBox([button, out])

close_button()

In [6]:
dir_labels = d["vc"][0.0][0]["params"]["dir_labels"][:]
null_idx = np.argwhere(dir_labels == 180)[0][0]
pref_idx = np.argwhere(dir_labels == 0)[0][0]
pos_ortho_idx = np.argwhere(dir_labels == 90)[0][0]
neg_ortho_idx = np.argwhere(dir_labels == 270)[0][0]

In [7]:
matching_states = True
for r in rhos:
    for i in idxs:
        if d["vc"][r][i]["params"]["rng_state"] != d["vc"][r][i]["params"]["rng_state"]:
            print("Seed mismatch at rho %s net %i" % (str(r), i))
            matching_states = False

if matching_states:
    print("All logged rng states match between vc and spk experiments.")

All logged rng states match between vc and spk experiments.


In [8]:
vc_key, g_key = "ACH", "E"
# vc_key, g_key = "GABA", "I"
vm_pts = d["spk"][0.0][0]["dendrites"]["g"]["E"].shape[-1]
for r in rhos:
    for i in idxs:
        vc_e = d["vc"][r][i]["dendrites"][vc_key]["g"][g_key][:, :, :, :vm_pts]
        vm_e = d["spk"][r][i]["dendrites"]["g"][g_key]
        diff = (vc_e - vm_e).sum()
        if diff != 0.0:
            print("ACH conductance off by %s at rho %s net %i" % (str(diff), str(r), i))

In [9]:
close_button()